In [38]:
from pyNN.utility import get_simulator, init_logging, normalized_filename
from pyNN.utility.plotting import Figure, Panel
import numpy as np
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN import space
import neo
from quantities import ms

import nmpi

c = nmpi.Client("nikhilgarg")
token = c.token
new_client = nmpi.Client("nikhilgarg", token=token)

In [39]:
# new_client.my_collabs()
# job_id = c.submit_job(source="/Users/Anonymous/Desktop/EMG/training.py",
#                       platform=nmpi.SPINNAKER,
#                       collab_id=85686)
# c.job_status(job_id)

In [40]:
import pyNN.spiNNaker as sim

In [41]:
seed = 764756387
rng = NumpyRNG(seed=seed, parallel_safe=True)

In [42]:
filename = 'EMG_dataset_with_spike_time_python3.npz'

In [35]:
data=np.load('EMG_dataset_with_spike_time_python3.npz',allow_pickle=True )

In [43]:
data.files

['X_EMG_Train',
 'Y_EMG_Train',
 'X_EMG_Test',
 'Y_EMG_Test',
 'spike_times_train_up',
 'spike_times_train_dn',
 'spike_times_test_up',
 'spike_times_test_dn']

In [37]:
X_EMG_Train = data['X_EMG_Train']
X_EMG_Test = data['X_EMG_Test']
Y_EMG_Train = data['Y_EMG_Train']
Y_EMG_Test = data['Y_EMG_Test']
spike_times_train_up = data['spike_times_train_up']
spike_times_train_dn = data['spike_times_train_dn']
spike_times_test_up = data['spike_times_test_up']
spike_times_test_dn = data['spike_times_test_dn']

In [44]:
n_exc = 256
n_inh = 64
n_stim = 8


cell_parameters = {
    'tau_m' : 20.0,# ms
    'tau_syn_E': 2.0, # ms
    'tau_syn_I': 5.0,# ms
    'v_rest': -65.0,  # mV
    'v_reset'  : -70.0,  # mV
    'v_thresh':  -50.0,# mV
    'cm':     1.0,    #nF
    'tau_refrac': 2.0,  # ms 
    'e_rev_E':   0.0,
    'e_rev_I': -70.0,
}
# cell_parameters = {'tau_refrac': 2.0,  # ms
#                'v_rest': -65.0,  # mV
#                 'v_reset'  : -70.0,  # mV
#                'v_thresh':  -50.0, # mV
#                'tau_syn_E':  2.0,  # ms
#                'tau_syn_I':  2.0,  # ms
#                'tau_m': 20
# }

synaptic_parameters = {
    'excitatory_stdp': {
        'timing_dependence': {'tau_plus': 20.0, 'tau_minus': 20.0,
                              'A_plus': 0.01, 'A_minus': 0.01},
        'weight_dependence': {'w_min': 0, 'w_max': 0.5},
        'weight': 0.1,
        'delay': 0.1},
    'excitatory_simple': {'weight': RandomDistribution('uniform', low=0.125, high=0.25, rng=rng),'delay': '0.1+0.001*d'},
    'inhibitory': {'weight': RandomDistribution('uniform', low= -0.25, high= -0.125, rng=rng), 'delay': '0.1+0.001*d'},
    'input': {'weight': 0.1, 'delay': 1},
}

In [45]:
connectivity_parameters = {
    'gaussian': {'d_expression': 'exp(-d**2/1e4)'},
    'global': {'p_connect': 0.1},
    'input_exc': {'n': 3},
    'input_inh': {'n': 2},
    'small_world' :{'degree': 10, 'n_connections': 20, 'rewiring':0.1, 'allow_self_connections':False },
}


In [46]:
grid_parameters = {
    'aspect_ratio': 1, 'dx': 50.0, 'dy': 50.0, 'fill_order': 'random'
}

In [47]:
class WeightRecorder(object):
    """
    Recording of weights is not yet built in to PyNN, so therefore we need
    to construct a callback object, which reads the current weights from
    the projection at regular intervals.
    """

    def __init__(self, sampling_interval, projection):
        self.interval = sampling_interval
        self.projection = projection
        self._weights = []

    def __call__(self, t):
        self._weights.append(self.projection.get('weight', format='list', with_address=False))
        return t + self.interval

    def get_weights(self):
        signal = neo.AnalogSignal(self._weights, units='nA', sampling_period=self.interval * ms,
                                  name="weight")
        signal.channel_index = neo.ChannelIndex(np.arange(len(self._weights[0])))
        return signal

Initiate the network

In [48]:
nb_samples = len(Y_EMG_Train)
nb_neurons = n_exc + n_inh
output_spike_count_matrix = np.zeros((nb_samples, nb_neurons))

In [49]:
for i in range(nb_samples) :
    print(i)
    input_ex = spike_times_train_up[1]
    input_inh = spike_times_train_dn[1]
   
    sim.setup(timestep=1)

    # all_cells = sim.Population(n_exc + n_inh, sim.IF_cond_exp(**cell_parameters),
    #                            structure=space.Grid2D(**grid_parameters),
    #                            label="All Cells")
    all_cells = sim.Population(n_exc + n_inh, sim.IF_cond_exp(**cell_parameters),
                               label="All Cells")
    exc_cells = all_cells[:n_exc]; exc_cells.label = "Excitatory cells"
    inh_cells = all_cells[n_exc:]; inh_cells.label = "Inhibitory cells"

    ext_stim_exc = sim.Population(n_stim, sim.SpikeSourceArray(spike_times= input_ex.tolist() ),
                              label="Input spike time array exitatory")
    ext_stim_inh = sim.Population(n_stim, sim.SpikeSourceArray(spike_times= input_inh.tolist() ),
                              label="Input spike time array inhibitory")

    
    gaussian_connectivity = sim.DistanceDependentProbabilityConnector(
                                 **connectivity_parameters['gaussian'])
    global_connectivity = sim.FixedProbabilityConnector(
                                **connectivity_parameters['global'])
    input_connectivity_exc = sim.FixedNumberPostConnector(
                                **connectivity_parameters['input_exc'])
    input_connectivity_inh = sim.FixedNumberPostConnector(
                                **connectivity_parameters['input_inh'])

    input_exc_exc_connections = sim.Projection(ext_stim_exc, exc_cells,
                                      input_connectivity_exc,
                                      receptor_type='excitatory',
                                      synapse_type=sim.StaticSynapse(**synaptic_parameters['input']),
                                      label='Input connections')
    input_inh_inh_connections = sim.Projection(ext_stim_inh, inh_cells,
                                      input_connectivity_inh,
                                      receptor_type='excitatory',
                                      synapse_type=sim.StaticSynapse(**synaptic_parameters['input']),
                                      label='Input connections')


    exc_exc_connections = sim.Projection(exc_cells, exc_cells,
                                     global_connectivity,
                                     receptor_type='excitatory',
                                     synapse_type=sim.StaticSynapse(**synaptic_parameters['excitatory_simple']),
                                     label='Excitatory - Excitatory connections')

    exc_inh_connections = sim.Projection(exc_cells, inh_cells,
                                     global_connectivity,
                                     receptor_type='excitatory',
                                     synapse_type=sim.StaticSynapse(**synaptic_parameters['excitatory_simple']),
                                     label='Excitatory - Inhibitory connections')

    inh_inh_connections = sim.Projection(inh_cells, exc_cells,
                                     global_connectivity,
                                     receptor_type='excitatory',
                                     synapse_type=sim.StaticSynapse(**synaptic_parameters['inhibitory']),
                                     label='Inhibitory connections')


    # == Instrument the network =================================================
#     weight_recorder_exc_exc= WeightRecorder(sampling_interval=0.1, projection=exc_exc_connections )
#     weight_recorder_exc_inh= WeightRecorder(sampling_interval=0.1, projection=exc_inh_connections )

    ext_stim_exc.record('spikes')
    ext_stim_inh.record('spikes')
    all_cells.record(['spikes', 'v'])

    # === Run the simulation =====================================================
    t_stop = 2000
    sim.run(2000)
    # sim.run(2000, callbacks=[weight_recorder_exc])

    #=== Store the simulation output into vatiables

    input_exc_data = ext_stim_exc.get_data().segments[0]
    input_inh_data = ext_stim_inh.get_data().segments[0]
    exc_cells_data = exc_cells.get_data().segments[0]
    inh_cells_data = inh_cells.get_data().segments[0]
#     weight_data = exc_exc_connections.getWeights()
#     weights_exc_exc = weight_recorder_exc_exc.get_weights()

    spike_count_vector = np.array(all_cells.get_spike_counts().values())

    output_spike_count_matrix[i] = spike_count_vector

0
Unable to find config file in any of the following locations: 
['/Library/Application Support/.spynnaker.cfg', '/Users/Anonymous/Library/Application Support/.spynnaker.cfg', '/Users/Anonymous/.spynnaker.cfg', './spynnaker.cfg']
********************************************************
/Users/Anonymous/.spynnaker.cfg has been created. 
Please edit this file and change "None" after "machineName" to the hostname or IP address of your SpiNNaker board, and change "None" after "version" to the version of SpiNNaker hardware you are running on:
[Machine]
machineName = None
version = None
***********************************************************



NoConfigFoundException: Unable to find config file in any of the following locations: 
['/Library/Application Support/.spynnaker.cfg', '/Users/Anonymous/Library/Application Support/.spynnaker.cfg', '/Users/Anonymous/.spynnaker.cfg', './spynnaker.cfg']
********************************************************
/Users/Anonymous/.spynnaker.cfg has been created. 
Please edit this file and change "None" after "machineName" to the hostname or IP address of your SpiNNaker board, and change "None" after "version" to the version of SpiNNaker hardware you are running on:
[Machine]
machineName = None
version = None
***********************************************************


In [1]:
output_spike_count_matrix

NameError: name 'output_spike_count_matrix' is not defined